In [1]:
import threading
import queue
import time
import random

# Shared queue between threads
buffer = queue.Queue(maxsize=5)

# Producer function
def producer():
    for i in range(10):
        item = f"item-{i}"
        buffer.put(item)
        print(f"Produced: {item}")
        time.sleep(random.random())
    buffer.put(None)  # Signal consumer to stop

# Consumer function
def consumer():
    while True:
        item = buffer.get()
        if item is None:
            break  # Stop signal
        print(f"Consumed: {item}")
        time.sleep(random.random())

# Threads creation
t1 = threading.Thread(target=producer)
t2 = threading.Thread(target=consumer)

# Start and wait for completion
t1.start()
t2.start()
t1.join()
t2.join()

Produced: item-0
Consumed: item-0
Produced: item-1
Consumed: item-1
Produced: item-2
Consumed: item-2
Produced: item-3
Produced: item-4
Produced: item-5
Consumed: item-3
Produced: item-6
Consumed: item-4
Consumed: item-5
Produced: item-7
Consumed: item-6
Consumed: item-7
Produced: item-8
Produced: item-9
Consumed: item-8
Consumed: item-9


In [14]:
import threading
import time

# Shared counter variable
counter = 0

def increment():
    """Increments the counter 100,000 times"""
    global counter
    for _ in range(100000):
        # This looks like one operation, but it's actually three:
        # 1. Read the current value of counter
        # 2. Add 1 to that value
        # 3. Write the new value back to counter
        counter += 1

# Create two threads that both increment the counter
thread1 = threading.Thread(target=increment)
thread2 = threading.Thread(target=increment)

# Start both threads
thread1.start()
thread2.start()

# Wait for both threads to complete
thread1.join()
thread2.join()

print(f"Final counter value: {counter}")
print(f"Expected value: 200000")
print(f"Lost updates: {200000 - counter}")

Final counter value: 200000
Expected value: 200000
Lost updates: 0


In [11]:
import threading
import time

# Shared counter variable
counter = 0

def increment():
    """Increments the counter with deliberate race conditions"""
    global counter
    for _ in range(10000):
        # Explicitly break the operation into steps to maximize race conditions
        temp = counter          # Step 1: Read current value
        temp = temp + 1         # Step 2: Increment
        time.sleep(0.00001)     # Step 3: CRITICAL - Force thread switch here!
        counter = temp          # Step 4: Write back (likely stale now)

# Run the demonstration
print("=" * 60)
print("RACE CONDITION DEMONSTRATION")
print("=" * 60)

# Create multiple threads for more contention
threads = []
num_threads = 5
increments_per_thread = 10000

print(f"\nStarting {num_threads} threads, each incrementing {increments_per_thread} times...")
print(f"Expected final value: {num_threads * increments_per_thread}")

start = time.time()

for i in range(num_threads):
    t = threading.Thread(target=increment)
    threads.append(t)
    t.start()

# Wait for all threads to complete
for t in threads:
    t.join()

end = time.time()

print(f"\nActual final value: {counter}")
print(f"Expected value: {num_threads * increments_per_thread}")
print(f"Lost updates: {num_threads * increments_per_thread - counter}")
print(f"Percentage lost: {((num_threads * increments_per_thread - counter) / (num_threads * increments_per_thread)) * 100:.1f}%")
print(f"Time taken: {end - start:.2f} seconds")

print("\n" + "=" * 60)
print("WHY THIS HAPPENS:")
print("=" * 60)
print("1. Thread A reads counter = 100")
print("2. Thread B reads counter = 100 (still!)")
print("3. Thread A writes counter = 101")
print("4. Thread B writes counter = 101 (overwrites A's update!)")
print("5. Result: Two increments, but counter only went up by 1")
print("\nThe sleep() forces context switches during the critical section,")
print("making this race condition almost guaranteed to occur.")

RACE CONDITION DEMONSTRATION

Starting 5 threads, each incrementing 10000 times...
Expected final value: 50000

Actual final value: 10001
Expected value: 50000
Lost updates: 39999
Percentage lost: 80.0%
Time taken: 0.71 seconds

WHY THIS HAPPENS:
1. Thread A reads counter = 100
2. Thread B reads counter = 100 (still!)
3. Thread A writes counter = 101
4. Thread B writes counter = 101 (overwrites A's update!)
5. Result: Two increments, but counter only went up by 1

The sleep() forces context switches during the critical section,
making this race condition almost guaranteed to occur.
